# Climpy data
- Reading a NetCDF file, analyzing the internals and finally extract the dessired layer into new files (under development)
- WIth GDAL, extracting the bands one by one into new files (finished)

In [1]:
import netCDF4 as nc
import numpy as np

In [4]:
"""Open the file and see what it is inside"""
try: fn.close()  # just to be safe, make sure dataset is not already open.
except: pass

# fn = r"//akif.internal/public/demo-geo-stack/geoserver/data/climpy_data/netcdf_examples/temperatura_superficial_agua_mar_cambios_sst_mediterraneo_rcp85_2081_2100.nc"
fn = r"//akif.internal/public/demo-geo-stack/geoserver/data/climpy_data/aries_mendia_climpy/climpy_rain_amount_corrected_copy.nc"
ds = nc.Dataset(fn, mode='r+') #, mode='w',format='NETCDF4'
# Here we print the characteristics
print(ds)
# here we print the characteristics as a dictionary
# print(ds.__dict__)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    CDI: Climate Data Interface version 1.9.9rc1 (https://mpimet.mpg.de/cdi)
    Conventions: CF-1.6
    history: Tue Dec 21 15:29:10 2021: cdo setmissval,-999 climpy_rain_amount_d_def.nc climpy_rain_amount_corrected.nc
Thu Dec 02 11:56:40 2021: cdo divc,10 climpy_rain_amount_df.nc climpy_rain_amount_d.nc
Thu Dec 02 10:58:18 2021: cdo setmissval,-999 climpy_rain_amount_def.nc climpy_rain_amount_df.nc
Wed Dec 01 14:11:24 2021: cdo divc,10 climpy_rain_amount_c.nc climpy_rain_amount_def.nc
Fri Nov 26 09:47:43 2021: cdo setmissval,-999 climpy_rain_amount.nc climpy_rain_amount_c.nc
    CDO: Climate Data Operators version 1.9.9rc1 (https://mpimet.mpg.de/cdo)
    dimensions(sizes): time(252), lon(553), lat(214)
    variables(dimensions): float64 time(time), float64 lon(lon), float64 lat(lat), float64 rain_amount(time,lat,lon)
    groups: 



c:\Users\admin\.conda\envs\gdal_env\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  


In [3]:
"""Here we check the dimensions: we get the three values, time, latitud and longitude"""
#longitude and latitute define the size of the layers. Time defines the number of layers inside 
for dim in ds.dimensions.values():
    print(dim) #it is no possible to iterate the dimension

"""Check variables"""
print (ds.variables.keys())



<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 252

<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 553

<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 214

odict_keys(['time', 'lon', 'lat', 'rain_amount'])


C:\Users\admin\.conda\envs\gdal\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  after removing the cwd from sys.path.


In [46]:
"""This is some testing. We want to get the time variable. So we are going to transform the values into an array"""
time_variable = ds.variables['time'] #we get the time variable
"""if we try to change the time variable, the rest of the data wil change too"""
# time_variable.units = "days since 1995-1-30"
print(time_variable.units) #here we get from which date it starts, the data officially starts at 1995, not at 1950. The content of this info may be edited by someone else.
print(time_variable.calendar)
print(time_variable.dimensions) # Here we get the dimensions in the variables

time_array = time_variable[:] #we get the array
print(time_array[0:3])
time_test = nc.num2date(time_array[0:3],"days since 1981-1-31")
for j in time_test:
    print(str(j))
    j_test = "_".join([str(s) for s in str(j).replace("-"," ").split() if s.isdigit()]) #get the date without the hh:mm:SS
    print(j_test)
print(time_test)


days since 1981-01-31
proleptic_gregorian
('time',)
[5113. 5141. 5172.]
1995-01-31 00:00:00
1995_01_31
1995-02-28 00:00:00
1995_02_28
1995-03-31 00:00:00
1995_03_31
[cftime.DatetimeGregorian(1995, 1, 31, 0, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(1995, 2, 28, 0, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(1995, 3, 31, 0, 0, 0, 0, has_year_zero=False)]


C:\Users\admin\.conda\envs\gdal\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  after removing the cwd from sys.path.
C:\Users\admin\.conda\envs\gdal\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  """
C:\Users\admin\.conda\envs\gdal\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


# How to read a certain part of the data

In [ ]:
import dateutil.parser

#concept of bands: https://stackoverflow.com/questions/52046282/convert-netcdf-nc-to-geotiff
sdt = dateutil.parser.parse("2015-07-20T00:00:00")
edt = dateutil.parser.parse("2015-07-24T00:00:00")

st_idx = nc.date2index(sdt, time_variable)
et_idx = nc.date2index(edt, time_variable)

"""We already defined the time variable"""
time_array_filtered = time_variable[st_idx:et_idx+1] #I want to read between 2015-07-20 00:00 to 2015-07-24 00:00
print(time_array_filtered)

# Here starts the real transformation. Netcdf to Geotif with GDAL

In [1]:
from osgeo import gdal
import netCDF4 as nc
import os

In [11]:
"""Here we get the number of bands and starting date"""

fn = r"//akif.internal/public/demo-geo-stack/geoserver/data/climpy_data/aries_mendia_climpy/climpy_rain_amount_corrected.nc"
ds = nc.Dataset(fn)

"""Set the dimension"""
dimension = 'time'

date_list = []

for dim in ds.dimensions.values():
    if dim == ds.dimensions[dimension]:
        number_bands = len(dim) #number of bands
        variable = ds.variables[dimension]
        starting_date = variable.units
        array = variable[:]
        time_format = nc.num2date(array,starting_date)
        for date_element in time_format:
            # replace "-" for " " so "split" can work, filter and create a list by digits and finally join them in a single string with "_" as separator
            date_element_formated = "_".join([str(s) for s in str(date_element).replace("-"," ").split() if s.isdigit()])
            date_list.append(date_element_formated)
    else:
        continue

print(number_bands)
print(date_list[0:3])
# print (ds.variables.keys())


252
['1995_01_31', '1995_02_28', '1995_03_31']


C:\Users\admin\.conda\envs\gdal\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  from ipykernel import kernelapp as app
C:\Users\admin\.conda\envs\gdal\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()


In [3]:
"""Here we create the output folder if it does not exist"""
root_dir = os.path.dirname(fn)
output_folder = "netcdf_output"
output_dir = os.path.join(root_dir, output_folder).replace("\\","/")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)



In [53]:
"""Some tests of naming files"""
file_name = os.path.splitext(fn)[0]
print(os.path.basename(fn))
print(file_name)

climpy_rain_amount_corrected_test.nc
//akif.internal/public/demo-geo-stack/geoserver/data/climpy_data/aries_mendia_climpy/climpy_rain_amount_corrected_test


In [12]:
"""Here we start transforming the data"""
gdal.UseExceptions()

# file_name = os.path.splitext(fn)[0]
print(os.path.basename(fn))
file_name = os.path.basename(fn)
k = 0
for i in date_list: # [0:2] "for testing"
    k += 1
    output = output_dir + "/" + file_name.replace(".nc","") + "_" + i + ".tif"
    """Method1"""
    # gdal.Translate(output, fn, 
    #                 format='GTIFF',
    #                 # format='NetCDF',
    #                 # bandList = [1]
    #                 # creationOptions = ['COMPRESS=DEFLATE','TILED=YES']
    #                 )
    """Method2"""
    gdaltranString = "gdal_translate -of GTiff -a_srs epsg:4326 -b " + str(k) + " -co COMPRESS=DEFLATE " + str(fn) + " " + str(output)
    os.system(gdaltranString)
    output = None

#gdal_translate -of GTiff -b 10 input.nc output.tiff  # to get 10th band, it works fine

climpy_rain_amount_corrected.nc
